In [9]:
import numpy as np

class AntColony:
    def __init__(self, num_ants, num_iterations, pheromone_decay, alpha, beta):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.pheromone_decay = pheromone_decay
        self.alpha = alpha
        self.beta = beta    
        self.ant_paths = []  # To store paths taken by each ant for each iteration

    def solve(self, problem):
        num_jobs = problem.num_jobs
        best_solution = None
        best_makespan = float('inf')

        for iteration in range(self.num_iterations):
            solutions = []
            iteration_paths = []  # To store paths taken by ants for this iteration
            for ant_id in range(self.num_ants):
                solution = self.construct_solution(problem)
                solutions.append(solution)
                iteration_paths.append(solution)  # Store the path taken by the ant
                self.ant_paths.append(iteration_paths)  # Store paths for each iteration

            for solution in solutions:
                makespan = problem.compute_makespan(solution)
                if makespan < best_makespan:
                    best_solution = solution
                    best_makespan = makespan

            self.update_pheromones(problem, solutions, best_solution)

            # Print paths taken by ants for this iteration
            print(f"Iteration {iteration + 1} - Paths taken by ants:")
            for i, path in enumerate(iteration_paths):
                print(f"Ant {i + 1}: {path}")

        return best_solution, best_makespan

    def construct_solution(self, problem):
        num_jobs = problem.num_jobs
        solution = np.zeros((num_jobs,), dtype=int)

        for job in range(num_jobs):
            remaining_jobs = [j for j in range(num_jobs) if j not in solution[:job]]
            probs = np.zeros((len(remaining_jobs),))

            for i, next_job in enumerate(remaining_jobs):
                if job == 0 or problem.can_execute_sequentially(solution[job - 1], next_job):
                    pheromone = problem.pheromones[solution[job - 1] if job > 0 else 0][next_job]
                    heuristic = 1 / problem.get_processing_time(job, next_job)  
                    probs[i] = (pheromone ** self.alpha) * (heuristic ** self.beta)

            if np.sum(probs) > 0:
                probs /= np.sum(probs)
                next_job_idx = np.random.choice(len(remaining_jobs), p=probs)
                next_job = remaining_jobs[next_job_idx]
                solution[job] = next_job

        return solution

    def update_pheromones(self, problem, solutions, best_solution):
        problem.pheromones *= (1 - self.pheromone_decay)
        for solution in solutions:
            for i, job in enumerate(solution):
                if i > 0:
                    problem.pheromones[solution[i - 1], job] += 1 / problem.compute_makespan(solution)
        for i, job in enumerate(best_solution):
            if i > 0:
                problem.pheromones[best_solution[i - 1], job] += 1 / problem.compute_makespan(best_solution)

class JobShopProblem:
    def __init__(self, processing_times):
        self.processing_times = processing_times 
        self.num_jobs, self.num_machines = processing_times.shape #rows - Jobs, cols - machines
        self.pheromones = np.ones((self.num_jobs, self.num_jobs))
        self.job_sequence = {0: [2, 1, 0], 1: [1, 3, 0], 2: [1, 0, 2]}  # Define job sequences

    def compute_makespan(self, solution):
        completion_times = np.zeros((self.num_jobs, self.num_machines))
        for job in range(self.num_jobs):
            machine = 0
            for operation in solution:
                processing_time = self.processing_times[job][operation]
                start_time = max(completion_times[job - 1][operation], completion_times[job][machine]) if job > 0 else 0
                completion_times[job][machine] = start_time + processing_time
                machine = (machine + 1) % self.num_machines
        return np.max(completion_times)

    def get_processing_time(self, job, next_job):
        return self.processing_times[job][next_job]

    def can_execute_sequentially(self, current_machine, next_job):
        # Check if the next job can be executed sequentially on the current machine
        if current_machine + 1 not in self.job_sequence:
            return False
        return next_job in self.job_sequence[current_machine + 1]
    

processing_times = np.array([[4, 3, 3],
                             [1, 2, 4],
                             [3, 2, 3]])
problem = JobShopProblem(processing_times)

num_ants = 3
num_iterations = 5
pheromone_decay = 0.1
alpha = 0.6
beta = 0.4

aco = AntColony(num_ants, num_iterations, pheromone_decay, alpha, beta)
best_solution, best_makespan = aco.solve(problem)

print("Best solution:", best_solution)
print("Best makespan:", best_makespan)


Iteration 1 - Paths taken by ants:
Ant 1: [2 0 1]
Ant 2: [0 1 2]
Ant 3: [0 1 2]
Iteration 2 - Paths taken by ants:
Ant 1: [2 0 1]
Ant 2: [1 0 0]
Ant 3: [0 1 2]
Iteration 3 - Paths taken by ants:
Ant 1: [0 1 2]
Ant 2: [2 0 1]
Ant 3: [1 0 0]
Iteration 4 - Paths taken by ants:
Ant 1: [2 0 1]
Ant 2: [0 1 2]
Ant 3: [2 0 1]
Iteration 5 - Paths taken by ants:
Ant 1: [2 0 1]
Ant 2: [1 0 0]
Ant 3: [1 0 0]
Best solution: [1 0 0]
Best makespan: 9.0
